## SCEdison

Explore usage and billing data.

Southern California Edison provides up to three projections of bill amount. Let's explore how accurate it is.

This is my personal billing info so it is a very small data set. None the less, it is fun to explore and can be used on larger data sets.


In [34]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np


In [30]:
# Load the billing and projections Excel sheets
xlsxpath = "E:/UserLo/source/repos/learning/SCEdison/"
xlsx = "SoCalEdison Bills.xlsx"
billing_sheet = "Bills"
projection_sheet = "Projections"

bills_df = pd.read_excel(xlsxpath+xlsx,sheet_name=billing_sheet)
projections_df = pd.read_excel(xlsxpath+xlsx,sheet_name=projection_sheet)

In [14]:
bills_df.head()

,month,day,year,date,amt,type,mid_peak_per_kwh,off_peak_per_kwh,super_off_peak_per_kwh,base_charge_per_day,base_credit_per_kwh,DWR_bond_per_kwh,RPV_tax_sub_total,st_tax_per_kwh,climate_credit
0,Apr,21,2022,2022-04-21,30.17,Bill,0.45312,0.34200,0.31000,0.031,-0.088545,0.00652,0.03,0.0003,-59.0
1,Mar,22,2022,2022-03-22,81.48,Bill,0.43767,0.33154,0.30128,0.031,-0.085275,0.00580,0.03,0.0003,NaN
2,Feb,18,2022,2022-02-18,72.73,Bill,0.41942,0.31647,0.28685,0.031,-0.082110,0.00580,0.03,0.0003,NaN
3,Jan,21,2022,2022-01-21,89.43,Bill,0.41479,0.31333,0.28355,0.031,-0.079910,0.00580,0.03,0.0003,NaN
4,Dec,20,2021,2021-12-20,90.26,Bill,0.40856,0.30843,0.27962,0.031,-0.079910,0.00580,0.03,0.0003,NaN


In [15]:
bills_df.dtypes

month                             object
day                                int64
year                               int64
date                      datetime64[ns]
amt                              float64
type                              object
mid_peak_per_kwh                 float64
off_peak_per_kwh                 float64
super_off_peak_per_kwh           float64
base_charge_per_day              float64
base_credit_per_kwh              float64
DWR_bond_per_kwh                 float64
RPV_tax_sub_total                float64
st_tax_per_kwh                   float64
climate_credit                   float64
dtype: object

In [7]:
projections_df.head()

,month,day,year,Date,projectedamt,remaining_days_in_billing_cycle
0,May,12,2022,2022-05-12,83.77,8
1,May,5,2022,2022-05-05,83.85,15
2,Apr,28,2022,2022-04-28,84.60,22
3,Apr,12,2022,2022-04-12,27.44,9
4,Apr,6,2022,2022-04-06,22.58,16


In [8]:
projections_df.dtypes

month                                      object
day                                         int64
year                                        int64
Date                               datetime64[ns]
projectedamt                              float64
remaining_days_in_billing_cycle             int64
dtype: object

Let's build a difference DataFrame, diff_df.

In [115]:
diff_df = bills_df[['date','amt']]

We need to build the interval of time for the particular billing cycle so we can find the projected amounts in each cycle.

In [116]:
diff_interval_s = pd.Series(data=[np.nan]*diff_df.index.stop,dtype='object',name='bill_interval')
for i,r in diff_df.iterrows():
    diff_interval_s.iloc[i] = pd.Interval(diff_df.iloc[i+1,0],r['date'])

    # Can't make interval on last row because there is no date previous to that one
    if i == (diff_df.index.stop)-2:
        break

diff_interval_s

0     (2022-03-22, 2022-04-21]
1     (2022-02-18, 2022-03-22]
2     (2022-01-21, 2022-02-18]
3     (2021-12-20, 2022-01-21]
4     (2021-11-19, 2021-12-20]
5     (2021-10-20, 2021-11-19]
6     (2021-09-20, 2021-10-20]
7     (2021-08-19, 2021-09-20]
8     (2021-07-21, 2021-08-19]
9     (2021-06-21, 2021-07-21]
10    (2021-05-20, 2021-06-21]
11    (2021-04-21, 2021-05-20]
12    (2021-03-22, 2021-04-21]
13    (2021-02-23, 2021-03-22]
14    (2021-01-23, 2021-02-23]
15                         NaN
Name: bill_interval, dtype: object

In [117]:
# Drop the last row with NaN interval
diff_df = pd.concat([diff_df,diff_interval_s],axis=1).dropna()
diff_df

,date,amt,bill_interval
0,2022-04-21,30.17,"(2022-03-22, 2022-04-21]"
1,2022-03-22,81.48,"(2022-02-18, 2022-03-22]"
2,2022-02-18,72.73,"(2022-01-21, 2022-02-18]"
3,2022-01-21,89.43,"(2021-12-20, 2022-01-21]"
4,2021-12-20,90.26,"(2021-11-19, 2021-12-20]"
5,2021-11-19,69.37,"(2021-10-20, 2021-11-19]"
6,2021-10-20,44.22,"(2021-09-20, 2021-10-20]"
7,2021-09-20,69.65,"(2021-08-19, 2021-09-20]"
8,2021-08-19,67.26,"(2021-07-21, 2021-08-19]"
9,2021-07-21,75.01,"(2021-06-21, 2021-07-21]"


In [84]:
# Find projected amounts for a bill interval. Let's try with one.
projections_df[projections_df['date'].map(lambda date: date in(diff_df.iloc[0]['bill_interval']))]

,month,day,year,date,projectedamt,remaining_days_in_billing_cycle
3,Apr,12,2022,2022-04-12,27.44,9
4,Apr,6,2022,2022-04-06,22.58,16
5,Mar,30,2022,2022-03-30,16.35,23


In [85]:
# Find most recent projected amount for a bill interval
projections_df[projections_df['date'].map(lambda date: date in(diff_df.iloc[0]['bill_interval']))].sort_values(by='date',ascending=False).iloc[0]['projectedamt']

27.44

Find the most recent projected amounts for each billing interval. For some intervals, I did not have any projected amounts so those are NaN.

In [118]:
most_recent_projected_amt_s = pd.Series(data=[np.nan]*len(diff_df.index),dtype='object',name='most_recent_projected_amt')
for i in diff_df.index:
    projections_in_interval_df = projections_df[projections_df['date'].map(lambda date: date in(diff_df.iloc[i]['bill_interval']))]
    if not projections_in_interval_df.empty:
        most_recent_projected_amt_s.iloc[i]= projections_in_interval_df.sort_values(by='date',ascending=False).iloc[0]['projectedamt']
diff_df = pd.concat([diff_df,most_recent_projected_amt_s],axis=1)
diff_df

,date,amt,bill_interval,most_recent_projected_amt
0,2022-04-21,30.17,"(2022-03-22, 2022-04-21]",27.44
1,2022-03-22,81.48,"(2022-02-18, 2022-03-22]",84.66
2,2022-02-18,72.73,"(2022-01-21, 2022-02-18]",74.28
3,2022-01-21,89.43,"(2021-12-20, 2022-01-21]",95.86
4,2021-12-20,90.26,"(2021-11-19, 2021-12-20]",86.92
5,2021-11-19,69.37,"(2021-10-20, 2021-11-19]",70.35
6,2021-10-20,44.22,"(2021-09-20, 2021-10-20]",NaN
7,2021-09-20,69.65,"(2021-08-19, 2021-09-20]",NaN
8,2021-08-19,67.26,"(2021-07-21, 2021-08-19]",NaN
9,2021-07-21,75.01,"(2021-06-21, 2021-07-21]",NaN


Now let's add an error column so we can compute some stats and generate some visualizations.

In [121]:
diff_df['error'] = diff_df['most_recent_projected_amt']-diff_df['amt']
diff_df

,date,amt,bill_interval,most_recent_projected_amt,error
0,2022-04-21,30.17,"(2022-03-22, 2022-04-21]",27.44,-2.73
1,2022-03-22,81.48,"(2022-02-18, 2022-03-22]",84.66,3.18
2,2022-02-18,72.73,"(2022-01-21, 2022-02-18]",74.28,1.55
3,2022-01-21,89.43,"(2021-12-20, 2022-01-21]",95.86,6.43
4,2021-12-20,90.26,"(2021-11-19, 2021-12-20]",86.92,-3.34
5,2021-11-19,69.37,"(2021-10-20, 2021-11-19]",70.35,0.98
6,2021-10-20,44.22,"(2021-09-20, 2021-10-20]",NaN,NaN
7,2021-09-20,69.65,"(2021-08-19, 2021-09-20]",NaN,NaN
8,2021-08-19,67.26,"(2021-07-21, 2021-08-19]",NaN,NaN
9,2021-07-21,75.01,"(2021-06-21, 2021-07-21]",NaN,NaN


### Not bad. Not bad.

The mean projected error is only $0.05. Some months are a little high. Some are a little low.

The absolute error is still pretty small: $2.65. With a mean absolute error percentage of 3.9%

In [135]:
print(f"Mean projected error: ${diff_df['error'].mean():1.2f}")
print(f"Mean projected absolute error: ${diff_df['error'].abs().mean():1.2f}")
print(f"Mean percent error: {(diff_df['error']/diff_df['amt']).mean():1.2%}")
print(f"Mean percent absolute error: {(diff_df['error']/diff_df['amt']).abs().mean():1.2%}")
print(f"Projected error variance: {diff_df['error'].var():1.2f}")

Mean projected error: $0.05
Mean projected absolute error: $2.65
Mean percent error: -0.64%
Mean percent absolute error: 3.90%
Projected error variance: 11.56


### Visualize the actuals and projections

Let's see the projected amounts compared to actual. Remember, you can have up to three projections before the actual bill.

In [33]:
fig = go.Figure()
fig.add_trace(go.Bar(x=bills_df['date'],
                y=bills_df['amt'],
                name='Actual Bill Amt',
                marker_color='darkblue'
                ))
fig.add_trace(go.Bar(x=projections_df['date'],
                y=projections_df['projectedamt'],
                name='Projected Bill Amt',
                marker_color='dodgerblue'
                ))

fig.update_layout(xaxis_tickangle=-90)

fig.show()

### Visualize the errors

The errors are pretty small so let's put them on a separate figure.

In [140]:
fig = go.Figure()

fig.add_trace(go.Bar(x=diff_df[diff_df['error']>0]['date'],
                y=diff_df[diff_df['error']>0]['error'],
                name='positive error',
                marker_color='black'
                ))
fig.add_trace(go.Bar(x=diff_df[diff_df['error']<0]['date'],
                y=diff_df[diff_df['error']<0]['error'],
                name='negative error',
                marker_color='red'
                ))

fig.update_layout(xaxis_tickangle=-90)

fig.show()

In [136]:
from plotly.subplots import make_subplots

### Combine into a plot with two subplots 

In [144]:
fig = make_subplots(rows=2,shared_xaxes='columns')

fig.add_trace(go.Bar(x=bills_df['date'],
                y=bills_df['amt'],
                name='Actual Bill Amt',
                marker_color='darkblue'
                ), row=1, col=1)
fig.add_trace(go.Bar(x=projections_df['date'],
                y=projections_df['projectedamt'],
                name='Projected Bill Amt',
                marker_color='dodgerblue'
                ), row=1, col=1)

fig.add_trace(go.Bar(x=diff_df[diff_df['error']>0]['date'],
                y=diff_df[diff_df['error']>0]['error'],
                name='positive error',
                marker_color='black'
                ), row=2, col=1)
fig.add_trace(go.Bar(x=diff_df[diff_df['error']<0]['date'],
                y=diff_df[diff_df['error']<0]['error'],
                name='negative error',
                marker_color='red'
                ), row=2, col=1)


fig.show()